In [1]:
#dependencies and point to where the data will live
import json
import pandas as pd
import numpy as np

file_dir = "C://Users/ryang/Documents/Bootcamp/Module8-ETL/movies-etl/Data/"

In [2]:
# load json data
with open(f'{file_dir}wikipedia-movies.json', mode = 'r') as file:
    wiki_movies_raw = json.load(file)

In [3]:
#load kaggle data
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [4]:
#check out wiki data

wiki_movies_df = pd.DataFrame(wiki_movies_raw)

len(wiki_movies_df.columns)

193

In [5]:
#filter out movies without a director or imdb link and add tv

wiki_movies = [movie for movie in wiki_movies_raw
                if ('Director' in movie or 'Directed by' in movie)
                and 'imdb_link' in movie
                and 'No. of episodes' not in movie]

len(wiki_movies)

7076

In [23]:
#starting the clean movie function

def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    
    #take all alternative titles, collapse to one column
    alt_titles = {}

    #loop through alt title names
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:

        #if any alt titles exist, add them to a dictionary, put that dictionary in movie list
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
        
        if len(alt_titles) > 0:
            movie['alt_titles'] = alt_titles
    
    #define a function to condense similar columns into one
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)

    #run above function on similar columns
    #see ColumnChanges.xlsx for work on deciding these columns
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Original language(s)', 'Language')
    change_column_name('Original release', 'Release date')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Voices of', 'Starring')
    change_column_name('Written by', 'Writer(s)')


    return movie

In [24]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

len(clean_movies)

7076

In [25]:
wiki_movies_df = pd.DataFrame(clean_movies)

sorted(wiki_movies_df.columns.to_list())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Narrated by',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']